In [13]:
import pandas as pd
import numpy as np
import datetime
import os
import seaborn as sns
import sqlalchemy
from sqlalchemy import create_engine

# OUTIL  IBM

## pour les Volumes

In [14]:
# création de la liste des fichiers à récupérer pour IBM volume

basepath_ibm_volume = 'C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/ibm/volume'
# List all files in a directory using scandir()
liste_ibm_volume=[]
with os.scandir(basepath_ibm_volume) as entries: # renvoie un itérateur
    for entry in entries:
        if entry.is_file():
            liste_ibm_volume.append(str(entry.name))
print(len(liste_ibm_volume))

898


In [15]:
# création du DF principal à partir de la liste_IBM_volume
df_ibm_volume = pd.DataFrame()
for name in liste_ibm_volume:
    with open(os.path.join(basepath_ibm_volume, name)) as f:
        df_temp = pd.read_fwf(f)
        df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
        df_ibm_volume = pd.concat([df_ibm_volume, df_temp])
        
#suppression des lignes où la used_capcity est null
df_ibm_volume = df_ibm_volume[df_ibm_volume.used_capacity.notnull()]        
df_ibm_volume

,vdisk_id,vdisk_name,copy_id,mdisk_grp_id,mdisk_grp_name,capacity,used_capacity,real_capacity,free_capacity,overallocation,...,grainsize,se_copy,compressed_copy,uncompressed_used_capacity,parent_mdisk_grp_id,parent_mdisk_grp_name,encrypt,deduplicated_copy,used_capacity_before_reduction,date
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,0,3,SDC3_T0,10.00GB,9.83GB,10.01GB,187.30MB,99,...,256.0,yes,no,9.83GB,3,SDC3_T0,yes,no,NaN,2018-10-03
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,1,2,SDC1_T0,10.00GB,9.83GB,10.01GB,184.75MB,99,...,256.0,yes,no,9.83GB,2,SDC1_T0,yes,no,NaN,2018-10-03
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,1,0,SDC1_T2,200.00GB,44.46GB,48.48GB,4.02GB,412,...,NaN,no,yes,157.05GB,0,SDC1_T2,yes,no,NaN,2018-10-03
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,1,0,SDC1_T2,200.00GB,44.47GB,48.50GB,4.03GB,412,...,NaN,no,yes,157.05GB,0,SDC1_T2,yes,no,NaN,2018-10-03
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,0,1,SDC3_T2,10.00GB,10.53GB,10.53GB,1.44MB,94,...,NaN,no,yes,9.82GB,1,SDC3_T2,yes,no,NaN,2018-10-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,768,CTZS_19DC1_T2M_CTZS_01_DS_02,1,7,SDC3_T2_FCM,3.00TB,149.96GB,211.41GB,61.45GB,1453,...,256.0,yes,no,149.96GB,7,SDC3_T2_FCM,yes,no,NaN,2021-03-18
934,769,CTZS_19DC3_T2M_CTZS_01_DS_02,0,7,SDC3_T2_FCM,3.00TB,177.25MB,82.12GB,81.95GB,3740,...,256.0,yes,no,177.25MB,7,SDC3_T2_FCM,yes,no,NaN,2021-03-18
935,769,CTZS_19DC3_T2M_CTZS_01_DS_02,1,6,SDC1_T2_FCM,3.00TB,177.25MB,82.11GB,81.93GB,3741,...,256.0,yes,no,177.25MB,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18
936,770,CR2N_19DC1_T2S_DS_01,0,6,SDC1_T2_FCM,256.00GB,166.41GB,171.54GB,5.13GB,149,...,256.0,yes,no,166.41GB,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18


In [16]:
# création d'une fonction qui recalcule les valeurs en MB et ne fait rien si la valeur null
import math
def en_MB(x):
    if type(x)==str:
        if "GB" in x :
            x = x.replace("GB","")
            x= float(x)*1024
        elif "G" in x:
            x = x.replace("G","")
            x= float(x)*1024
        elif "MB" in x:
            x = x.replace("MB","")
            x= float(x)
        elif "TB" in x:
            x = x.replace("TB","")
            x = float(x)*1024*1024
    elif math.isnan(float(x)):
        pass
    return x

In [17]:
# transformation des colonnes capacity, used_capacity,real_capacity, free_capacity, uncompressed_used_capacity, used_capacity_before_reduction
for colonne in ["capacity", "used_capacity","real_capacity", "free_capacity", "uncompressed_used_capacity", "used_capacity_before_reduction"]:
    df_ibm_volume[colonne] = df_ibm_volume[colonne].map(lambda x : en_MB(x) )
df_ibm_volume

,vdisk_id,vdisk_name,copy_id,mdisk_grp_id,mdisk_grp_name,capacity,used_capacity,real_capacity,free_capacity,overallocation,...,grainsize,se_copy,compressed_copy,uncompressed_used_capacity,parent_mdisk_grp_id,parent_mdisk_grp_name,encrypt,deduplicated_copy,used_capacity_before_reduction,date
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,0,3,SDC3_T0,10240.0,10065.92,10250.24,187.30,99,...,256.0,yes,no,10065.92,3,SDC3_T0,yes,no,NaN,2018-10-03
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,1,2,SDC1_T0,10240.0,10065.92,10250.24,184.75,99,...,256.0,yes,no,10065.92,2,SDC1_T0,yes,no,NaN,2018-10-03
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,1,0,SDC1_T2,204800.0,45527.04,49643.52,4116.48,412,...,NaN,no,yes,160819.20,0,SDC1_T2,yes,no,NaN,2018-10-03
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,1,0,SDC1_T2,204800.0,45537.28,49664.00,4126.72,412,...,NaN,no,yes,160819.20,0,SDC1_T2,yes,no,NaN,2018-10-03
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,0,1,SDC3_T2,10240.0,10782.72,10782.72,1.44,94,...,NaN,no,yes,10055.68,1,SDC3_T2,yes,no,NaN,2018-10-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,768,CTZS_19DC1_T2M_CTZS_01_DS_02,1,7,SDC3_T2_FCM,3145728.0,153559.04,216483.84,62924.80,1453,...,256.0,yes,no,153559.04,7,SDC3_T2_FCM,yes,no,NaN,2021-03-18
934,769,CTZS_19DC3_T2M_CTZS_01_DS_02,0,7,SDC3_T2_FCM,3145728.0,177.25,84090.88,83916.80,3740,...,256.0,yes,no,177.25,7,SDC3_T2_FCM,yes,no,NaN,2021-03-18
935,769,CTZS_19DC3_T2M_CTZS_01_DS_02,1,6,SDC1_T2_FCM,3145728.0,177.25,84080.64,83896.32,3741,...,256.0,yes,no,177.25,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18
936,770,CR2N_19DC1_T2S_DS_01,0,6,SDC1_T2_FCM,262144.0,170403.84,175656.96,5253.12,149,...,256.0,yes,no,170403.84,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18


In [18]:
# renommage des colonnes
df_ibm_volume = df_ibm_volume.rename(columns={"vdisk_id":"ID","vdisk_name":"name","mdisk_grp_name": "pool"})

#création d'une colonne offre
df_ibm_volume['offre'] = df_ibm_volume.pool
df_ibm_volume.offre= df_ibm_volume.offre.map(lambda x: x[5:])

# création d'une colonne origine
df_ibm_volume["origine"] = np.array("ibm")
df_ibm_volume

,ID,name,copy_id,mdisk_grp_id,pool,capacity,used_capacity,real_capacity,free_capacity,overallocation,...,compressed_copy,uncompressed_used_capacity,parent_mdisk_grp_id,parent_mdisk_grp_name,encrypt,deduplicated_copy,used_capacity_before_reduction,date,offre,origine
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,0,3,SDC3_T0,10240.0,10065.92,10250.24,187.30,99,...,no,10065.92,3,SDC3_T0,yes,no,NaN,2018-10-03,T0,ibm
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,1,2,SDC1_T0,10240.0,10065.92,10250.24,184.75,99,...,no,10065.92,2,SDC1_T0,yes,no,NaN,2018-10-03,T0,ibm
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,1,0,SDC1_T2,204800.0,45527.04,49643.52,4116.48,412,...,yes,160819.20,0,SDC1_T2,yes,no,NaN,2018-10-03,T2,ibm
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,1,0,SDC1_T2,204800.0,45537.28,49664.00,4126.72,412,...,yes,160819.20,0,SDC1_T2,yes,no,NaN,2018-10-03,T2,ibm
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,0,1,SDC3_T2,10240.0,10782.72,10782.72,1.44,94,...,yes,10055.68,1,SDC3_T2,yes,no,NaN,2018-10-03,T2,ibm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,768,CTZS_19DC1_T2M_CTZS_01_DS_02,1,7,SDC3_T2_FCM,3145728.0,153559.04,216483.84,62924.80,1453,...,no,153559.04,7,SDC3_T2_FCM,yes,no,NaN,2021-03-18,T2_FCM,ibm
934,769,CTZS_19DC3_T2M_CTZS_01_DS_02,0,7,SDC3_T2_FCM,3145728.0,177.25,84090.88,83916.80,3740,...,no,177.25,7,SDC3_T2_FCM,yes,no,NaN,2021-03-18,T2_FCM,ibm
935,769,CTZS_19DC3_T2M_CTZS_01_DS_02,1,6,SDC1_T2_FCM,3145728.0,177.25,84080.64,83896.32,3741,...,no,177.25,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18,T2_FCM,ibm
936,770,CR2N_19DC1_T2S_DS_01,0,6,SDC1_T2_FCM,262144.0,170403.84,175656.96,5253.12,149,...,no,170403.84,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18,T2_FCM,ibm


In [19]:
df_ibm_volume_reduit = df_ibm_volume[["ID","name","pool","offre","capacity","used_capacity",'date','origine']].copy()

# réinitialise les index
df_ibm_volume_reduit.reset_index(drop=True,inplace =True)
df_ibm_volume_reduit

,ID,name,pool,offre,capacity,used_capacity,date,origine
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC3_T0,T0,10240.0,10065.92,2018-10-03,ibm
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC1_T0,T0,10240.0,10065.92,2018-10-03,ibm
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2,T2,204800.0,45527.04,2018-10-03,ibm
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2,T2,204800.0,45537.28,2018-10-03,ibm
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,SDC3_T2,T2,10240.0,10782.72,2018-10-03,ibm
...,...,...,...,...,...,...,...,...
633891,768,CTZS_19DC1_T2M_CTZS_01_DS_02,SDC3_T2_FCM,T2_FCM,3145728.0,153559.04,2021-03-18,ibm
633892,769,CTZS_19DC3_T2M_CTZS_01_DS_02,SDC3_T2_FCM,T2_FCM,3145728.0,177.25,2021-03-18,ibm
633893,769,CTZS_19DC3_T2M_CTZS_01_DS_02,SDC1_T2_FCM,T2_FCM,3145728.0,177.25,2021-03-18,ibm
633894,770,CR2N_19DC1_T2S_DS_01,SDC1_T2_FCM,T2_FCM,262144.0,170403.84,2021-03-18,ibm


In [20]:
# fonction qui convertit en GB
def en_giga(x):
    return x/1024

In [21]:
# transformation des colonnes capacity, used_capacity,real_capacity, free_capacity, uncompressed_used_capacity, used_capacity_before_reduction

df_ibm_volume_GB = df_ibm_volume_reduit.copy()
for colonne in ["capacity", "used_capacity"]:
     df_ibm_volume_GB[colonne] = df_ibm_volume_reduit[colonne].map(lambda x : en_giga(x)).copy()
df_ibm_volume_GB

,ID,name,pool,offre,capacity,used_capacity,date,origine
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC3_T0,T0,10.0,9.830000,2018-10-03,ibm
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC1_T0,T0,10.0,9.830000,2018-10-03,ibm
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2,T2,200.0,44.460000,2018-10-03,ibm
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2,T2,200.0,44.470000,2018-10-03,ibm
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,SDC3_T2,T2,10.0,10.530000,2018-10-03,ibm
...,...,...,...,...,...,...,...,...
633891,768,CTZS_19DC1_T2M_CTZS_01_DS_02,SDC3_T2_FCM,T2_FCM,3072.0,149.960000,2021-03-18,ibm
633892,769,CTZS_19DC3_T2M_CTZS_01_DS_02,SDC3_T2_FCM,T2_FCM,3072.0,0.173096,2021-03-18,ibm
633893,769,CTZS_19DC3_T2M_CTZS_01_DS_02,SDC1_T2_FCM,T2_FCM,3072.0,0.173096,2021-03-18,ibm
633894,770,CR2N_19DC1_T2S_DS_01,SDC1_T2_FCM,T2_FCM,256.0,166.410000,2021-03-18,ibm


In [22]:
## Pour avoir la typologie des volumes , on crée à partir d'un fichier IBM server un DF liant le nom du volume
#au server qui l'héberge

In [23]:
# Fonction qui trouve la typologie du server
def typo(serveur):
    if "APP" in serveur:
        typo = "APP"
    elif "WEB" in serveur:
        typo = "WEB"
    elif "DAT" in serveur:
        typo = "DAT"
    elif "SQL" in serveur:
        typo = "DAT"
    elif "ORA" in serveur:
        typo = "DAT"
    elif "VIR" in serveur:
        typo = "VIR"
    elif "vir" in serveur:
        typo = "VIR"
    elif "FIC" in serveur:
        typo = "FIC"
    elif "MES" in serveur:
        typo = "MES"
    elif "DNS" in serveur:
        typo = "DNS"
    else:
        typo = "AUTRE"
 
    return typo

In [24]:
# création du DF Server volume avec un fichier server IBM (le plus récent) 
# taille du fichier 236 000 ko.
basepath='C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/ibm/server/'
df_ibm_server = pd.DataFrame()
for name in ["20210317_mutsapsan019_lshostvdiskmap.txt"]:
    with open(os.path.join(basepath, name)) as f:
        df_temp = pd.read_csv(f, delim_whitespace =True, names =['ID', 'name_server',"SCSI_id" ,"vdisk_id" ,"vdisk_name" ,"vdisk_UID","IO_group_id","IO_group_name", "mapping_type","host_cluster_id","host_cluster_name"])
        df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
        df_ibm_server = pd.concat([df_ibm_server, df_temp])
        
df_ibm_server.reset_index(drop=True,inplace =True)
# supprime les lignes où le name n'est pas bon.
df_ibm_server = df_ibm_server.drop(df_ibm_server.name_server[df_ibm_server.name_server == "name"].index)
df_ibm_server = df_ibm_server.drop(df_ibm_server.name_server[df_ibm_server.vdisk_name.isnull()].index)

# création d'une colonne typo
df_ibm_server['typologie'] = df_ibm_server.name_server
df_ibm_server.typologie = df_ibm_server.typologie.map(lambda x : typo(x))
df_ibm_server

,ID,name_server,SCSI_id,vdisk_id,vdisk_name,vdisk_UID,IO_group_id,IO_group_name,mapping_type,host_cluster_id,host_cluster_name,date,typologie
2,0,EXLIDAT1,0,428,INFN_19DC3_T2M_EXLIDAT1_TEST_01,600507680C80856BD8000000000001D0,0,io_grp0,private,NaN,NaN,2021-03-17,DAT
3,0,EXLIDAT1,3,12,MUTN_19DC1_T2S_BACULA_CATALOGUE_01,600507680C80856BD80000000000000E,0,io_grp0,private,NaN,NaN,2021-03-17,DAT
4,0,EXLIDAT1,1,11,MUTN_19DC1_T2S_BACULA_PGSQL_01,600507680C80856BD80000000000000D,1,io_grp1,private,NaN,NaN,2021-03-17,DAT
7,1,EXLIDAT2,1,10,MUTN_19DC3_T2S_BACULA_PGSQL_01,600507680C80856BD800000000000014,0,io_grp0,private,NaN,NaN,2021-03-17,DAT
8,1,EXLIDAT2,3,9,MUTN_19DC3_T2S_BACULA_CATALOGUE_01,600507680C80856BD800000000000013,1,io_grp1,private,NaN,NaN,2021-03-17,DAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591979,204,PICSVPVIR002,3,750,PICS_19DC3_T2M_PICS_01_DS_01,600507680C80856BD800000000000880,2,io_grp2,shared,55,CL_CLOUD_P_PICS_01,2021-03-17,VIR
1591982,206,PRVNVPVIR002,1,754,PRVN_19DC3_T2M_PRVN_01_DS_01,600507680C80856BD800000000000886,1,io_grp1,shared,56,CL_CLOUD_P_PRVN_01,2021-03-17,VIR
1591983,206,PRVNVPVIR002,0,753,PRVN_19DC1_T2M_PRVN_01_DS_01,600507680C80856BD800000000000885,2,io_grp2,shared,56,CL_CLOUD_P_PRVN_01,2021-03-17,VIR
1591986,208,PRVNVPVIR001,1,754,PRVN_19DC3_T2M_PRVN_01_DS_01,600507680C80856BD800000000000886,1,io_grp1,shared,56,CL_CLOUD_P_PRVN_01,2021-03-17,VIR


In [25]:
#Création d'un df pour connaitre la typologie du volume et le nom de client(si possible) à partir d'un nom de server associé
liste_volume = []
liste_type = []
liste_client =[]
for volume in df_ibm_server.vdisk_name.unique():
    if volume in liste_volume:
        pass
    else:
        liste_volume.append(volume)
        liste_client.append(df_ibm_server[df_ibm_server.vdisk_name == volume].name_server.unique()[0][:3])
        liste_type.append(df_ibm_server[df_ibm_server.vdisk_name == volume].typologie.unique()[0])
df_t = pd.DataFrame({"name_v":liste_volume,"client":liste_client,"type":liste_type})
df_t

,name_v,client,type
0,INFN_19DC3_T2M_EXLIDAT1_TEST_01,EXL,DAT
1,MUTN_19DC1_T2S_BACULA_CATALOGUE_01,EXL,DAT
2,MUTN_19DC1_T2S_BACULA_PGSQL_01,EXL,DAT
3,MUTN_19DC3_T2S_BACULA_PGSQL_01,EXL,DAT
4,MUTN_19DC3_T2S_BACULA_CATALOGUE_01,EXL,DAT
...,...,...,...
1031,TEMP_INFSVPVIR030_SDC1_T2,INF,VIR
1032,CTZS_19DC1_T2M_CTZS_01_DS_02,CTZ,VIR
1033,CTZS_19DC3_T2M_CTZS_01_DS_02,CTZ,VIR
1034,CR2N_19DC1_T2S_DS_01,MUT,VIR


In [26]:
# création du DF Server volmue avec un fichier server IBM (le plus ancien) 
# taille du fichier 43 000 ko.

df_ibm_server_old = pd.DataFrame()
for name in ["20181003_mutsapsan019_lshostvdiskmap.txt"]:
    with open(os.path.join(basepath, name)) as f:
        df_temp = pd.read_csv(f, delim_whitespace =True, names =['ID', 'name_server',"SCSI_id" ,"vdisk_id" ,"vdisk_name" ,"vdisk_UID","IO_group_id","IO_group_name", "mapping_type","host_cluster_id","host_cluster_name"])
        df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
        df_ibm_server_old = pd.concat([df_ibm_server_old, df_temp])


# supprime les lignes où le name n'est pas bon.
df_ibm_server_old = df_ibm_server_old.drop(df_ibm_server_old.name_server[df_ibm_server_old.name_server == "name"].index)
df_ibm_server_old = df_ibm_server_old.drop(df_ibm_server_old.name_server[df_ibm_server_old.vdisk_name.isnull()].index)

# création d'une colonne typo
df_ibm_server_old['typologie'] = df_ibm_server_old.name_server
df_ibm_server_old.typologie = df_ibm_server_old.typologie.map(lambda x : typo(x))
df_ibm_server_old

,ID,name_server,SCSI_id,vdisk_id,vdisk_name,vdisk_UID,IO_group_id,IO_group_name,mapping_type,host_cluster_id,host_cluster_name,date,typologie
2,0,EXLIDAT1,0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,600507680C80856BD800000000000000,0,io_grp0,private,NaN,NaN,2018-10-03,DAT
3,0,EXLIDAT1,3,12,MUTN_19DC1_T2S_BACULA_CATALOGUE_01,600507680C80856BD80000000000000E,0,io_grp0,private,NaN,NaN,2018-10-03,DAT
4,0,EXLIDAT1,0,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,600507680C80856BD800000000000005,1,io_grp1,private,NaN,NaN,2018-10-03,DAT
5,0,EXLIDAT1,1,11,MUTN_19DC1_T2S_BACULA_PGSQL_01,600507680C80856BD80000000000000D,1,io_grp1,private,NaN,NaN,2018-10-03,DAT
8,1,EXLIDAT2,0,6,INFS_19DC3_T2M_TEST_EXLIDAT2_01,600507680C80856BD800000000000008,0,io_grp0,private,NaN,NaN,2018-10-03,DAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,89,IPSSVPVIR016,2,236,IPSS_19DC3_T2M_DS_03,600507680C80856BD8000000000000FE,0,io_grp0,shared,11,CL_CLOUD_P_IPSS_02,2018-10-03,VIR
2223,89,IPSSVPVIR016,0,231,IPSS_19DC1_T2M_DS_01,600507680C80856BD8000000000000F9,1,io_grp1,shared,11,CL_CLOUD_P_IPSS_02,2018-10-03,VIR
2224,89,IPSSVPVIR016,1,233,IPSS_19DC1_T2M_DS_03,600507680C80856BD8000000000000FB,1,io_grp1,shared,11,CL_CLOUD_P_IPSS_02,2018-10-03,VIR
2225,89,IPSSVPVIR016,2,235,IPSS_19DC3_T2M_DS_02,600507680C80856BD8000000000000FD,1,io_grp1,shared,11,CL_CLOUD_P_IPSS_02,2018-10-03,VIR


In [27]:
#Création d'un df pour connaitre la typologie du volume et le nom de client(si possible) à partir d'un nom de server associé
# idem mais avec le fichier le plus ancien.
liste_volume_old = []
liste_type_old = []
liste_client_old =[]
for volume in df_ibm_server_old.vdisk_name.unique():
    if volume in liste_volume_old:
        pass
    else:
        liste_volume_old.append(volume)
        liste_client_old.append(df_ibm_server_old[df_ibm_server_old.vdisk_name == volume].name_server.unique()[0][:3])
        liste_type_old.append(df_ibm_server_old[df_ibm_server_old.vdisk_name == volume].typologie.unique()[0])
df_t_old = pd.DataFrame({"name_v":liste_volume_old,"client":liste_client_old,"type":liste_type_old})
df_t_old

,name_v,client,type
0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,EXL,DAT
1,MUTN_19DC1_T2S_BACULA_CATALOGUE_01,EXL,DAT
2,INFS_19DC1_T2M_TEST_EXLIDAT1_01,EXL,DAT
3,MUTN_19DC1_T2S_BACULA_PGSQL_01,EXL,DAT
4,INFS_19DC3_T2M_TEST_EXLIDAT2_01,EXL,DAT
...,...,...,...
377,LNAS_19DC3_T2S_ORALNAP_DATA_FG2_01,LNA,DAT
378,LNAS_19DC3_T2S_ORALNAP_DATA_FG2_03,LNA,DAT
379,A24N_19DC1_T2S_U01_2,NAL,WEB
380,A24N_19DC1_T2S_U01_1,NAL,WEB


In [28]:
print("nb de client",len(df_t_old.client.unique()))
# nb de volumes par clients reconnus
df_t_old.client.value_counts()

nb de client 14


MUT    114
ELE    106
USG     41
LNA     26
CMC     22
PXP     22
INF     20
EXL      8
IPS      7
ORA      5
HAR      4
NAL      3
GAM      3
POL      1
Name: client, dtype: int64

In [29]:
compt=0
for v in df_t.name_v:
    if v not in df_t_old.name_v.to_list():
        compt+=1
print("nb de nouveaux volumes ",compt)
compt=0
for v in df_t_old.name_v:
    if v not in df_t.name_v.to_list():
        compt+=1
print("nb d'anciens volumes qui auraient disparus",compt)

nb de nouveaux volumes  700
nb d'anciens volumes qui auraient disparus 46


In [30]:
liste_non_typé=[]
for v in df_ibm_volume_GB.name.unique():
    if v not in df_t[df_t.name_v == v].name_v.unique():
        liste_non_typé.append(v)
print('nb de volumes distincts',len(df_ibm_volume_GB.name.unique()))
print('nb de volumes typés',len(df_ibm_volume_GB.name.unique())-len(liste_non_typé))
print('nb de volumes non typés',len(liste_non_typé))

nb de volumes distincts 1309
nb de volumes typés 1023
nb de volumes non typés 286


In [31]:
# Etiquetage en typologie et nom des clients des volumes ibm à partir du dataframe constitué précédemment df_t.
# Si le serveur n'est reconnu, on tente de trouver la typo via son nom avec la fonction typo()
df_ibm_volume_plus = df_ibm_volume_GB.copy()

liste_typo =[]
liste_client=[]

for v in df_ibm_volume_plus.name:
    if v in df_t.name_v.unique().tolist(): # récupération de la typologie via le df df_t
        liste_typo.append(df_t[df_t.name_v == v].type.values[0])
        liste_client.append(df_t[df_t.name_v == v].client.values[0])
    else:
        if v[:3]=="INF":
            liste_typo.append(typo(v))   # appel de la fonction typo() pour trouver la typologie
            liste_client.append('INF')
        elif v[:3]=="SIG":
            liste_typo.append(typo(v))   
            liste_client.append('SIG')
        else :
            liste_typo.append(typo(v)) 
            liste_client.append('Client_non_enregistré')
df_ibm_volume_plus['typologie'] = liste_typo
df_ibm_volume_plus['client'] = liste_client
df_ibm_volume_plus

,ID,name,pool,offre,capacity,used_capacity,date,origine,typologie,client
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC3_T0,T0,10.0,9.830000,2018-10-03,ibm,DAT,INF
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC1_T0,T0,10.0,9.830000,2018-10-03,ibm,DAT,INF
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2,T2,200.0,44.460000,2018-10-03,ibm,DAT,CMC
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2,T2,200.0,44.470000,2018-10-03,ibm,DAT,CMC
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,SDC3_T2,T2,10.0,10.530000,2018-10-03,ibm,DAT,INF
...,...,...,...,...,...,...,...,...,...,...
633891,768,CTZS_19DC1_T2M_CTZS_01_DS_02,SDC3_T2_FCM,T2_FCM,3072.0,149.960000,2021-03-18,ibm,VIR,CTZ
633892,769,CTZS_19DC3_T2M_CTZS_01_DS_02,SDC3_T2_FCM,T2_FCM,3072.0,0.173096,2021-03-18,ibm,VIR,CTZ
633893,769,CTZS_19DC3_T2M_CTZS_01_DS_02,SDC1_T2_FCM,T2_FCM,3072.0,0.173096,2021-03-18,ibm,VIR,CTZ
633894,770,CR2N_19DC1_T2S_DS_01,SDC1_T2_FCM,T2_FCM,256.0,166.410000,2021-03-18,ibm,VIR,MUT


In [32]:
## Enregistrement 
df_ibm_volume_plus.to_csv("df_ibm_volume_plus.csv", index=False)

## Récupération du csv pour poursuite des analyses

In [33]:
df_ibm_month = df_ibm_volume_plus.copy()
liste_année_mois = []
for i in df_ibm_month.date:
    liste_année_mois.append(str(i)[:7])
df_ibm_month['année_mois'] =liste_année_mois
df_ibm_month

,ID,name,pool,offre,capacity,used_capacity,date,origine,typologie,client,année_mois
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC3_T0,T0,10.0,9.830000,2018-10-03,ibm,DAT,INF,2018-10
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC1_T0,T0,10.0,9.830000,2018-10-03,ibm,DAT,INF,2018-10
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2,T2,200.0,44.460000,2018-10-03,ibm,DAT,CMC,2018-10
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2,T2,200.0,44.470000,2018-10-03,ibm,DAT,CMC,2018-10
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,SDC3_T2,T2,10.0,10.530000,2018-10-03,ibm,DAT,INF,2018-10
...,...,...,...,...,...,...,...,...,...,...,...
633891,768,CTZS_19DC1_T2M_CTZS_01_DS_02,SDC3_T2_FCM,T2_FCM,3072.0,149.960000,2021-03-18,ibm,VIR,CTZ,2021-03
633892,769,CTZS_19DC3_T2M_CTZS_01_DS_02,SDC3_T2_FCM,T2_FCM,3072.0,0.173096,2021-03-18,ibm,VIR,CTZ,2021-03
633893,769,CTZS_19DC3_T2M_CTZS_01_DS_02,SDC1_T2_FCM,T2_FCM,3072.0,0.173096,2021-03-18,ibm,VIR,CTZ,2021-03
633894,770,CR2N_19DC1_T2S_DS_01,SDC1_T2_FCM,T2_FCM,256.0,166.410000,2021-03-18,ibm,VIR,MUT,2021-03


In [34]:
df_ibm_volume_plus = pd.read_csv("df_ibm_volume_plus.csv")

# 3 PAR

 ## Pour les volumes

In [35]:
# création de la liste des fichiers à récupérer pour 3par volume

basepath_3par_volume = 'C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/3par/volume'
# List all files in a directory using scandir()
liste_3par_volume=[]
with os.scandir(basepath_3par_volume) as entries: # renvoie un itérateur
    for entry in entries:
        if entry.is_file():
            liste_3par_volume.append(str(entry.name))
print(len(liste_3par_volume))

2250


In [36]:
# création du DF principal à partir de la liste_3par_volume

df_3par_volume = pd.DataFrame()
for name in liste_3par_volume:
    with open(os.path.join(basepath_3par_volume, name)) as f:
        # gestion des formats de fichiers avant / après le 08 janvier 2020
        if datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')<= datetime.datetime(2020,1,8):
            df_temp = pd.read_csv(f,delim_whitespace =True,  names= ["ID","name","Prov","Type","adm_Rsvd","adm_Used","snp_Rsvd","snp_Used","%_snp_Used","snp_Wrn","snp_Lim","usr_Rsvd" ,"usr_Used","%usr_Used", "usr_Wrn" ,"usr_Lim","Tot_Rsvd","VSize","Compaction","Dedup"], header = 4)
            df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
            df_temp['pool']= name[9:19]
            df_temp = df_temp.rename(columns={"Tot_Rsvd": "Tot_used"})
        else:
            df_temp = pd.read_csv(f,delim_whitespace =True,  names= ["ID","name","Prov","Type","adm_Rsvd","adm_Used","snp_Rsvd","snp_Used","%_snp_Used","snp_Wrn","snp_Lim","usr_Rsvd" ,"usr_Used","%usr_Used", "usr_Wrn" ,"usr_Lim","Tot_Rsvd","Tot_used","HostWR","VSize","Compact","Compress"], header = 4)
            df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
            df_temp['pool']= name[9:19]
        df_3par_volume = pd.concat([df_3par_volume, df_temp])
        
# supprime les lignes où le name n'est pas bon.        
df_3par_volume = df_3par_volume.drop(df_3par_volume.name[df_3par_volume.name.isnull()].index)
df_3par_volume = df_3par_volume.drop(df_3par_volume.name[df_3par_volume.name=='total'].index)
df_3par_volume = df_3par_volume[df_3par_volume.name !="No"]
df_3par_volume

,ID,name,Prov,Type,adm_Rsvd,adm_Used,snp_Rsvd,snp_Used,%_snp_Used,snp_Wrn,...,Tot_used,VSize,Compaction,Dedup,date,pool,Tot_Rsvd,HostWR,Compact,Compress
0,70,GTGN_05T2N_DS_GTG_DATA_02,tpvv,base,1664,1418,0.0,0,0,--,...,2301696.0,2301952.0,1,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
1,69,GTGN_05T2N_DS_GTG_OS_02,tpvv,base,256,51,512.0,0,0,0,...,73984.0,512000.0,7.5,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
2,73,IAXI_05T2S_OVM-12CR2_01,tpvv,base,256,21,512.0,0,0,0,...,32000.0,30720.0,1,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
3,44,IAXI_05T2S_OVM-RAC01_01,tpvv,base,256,15,0.0,0,0,--,...,21248.0,25600.0,1.3,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
4,45,IAXI_05T2S_OVM-RAC01_02,tpvv,base,256,15,0.0,0,0,--,...,21248.0,25600.0,1.3,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,78,SIGN_06DC3_T2S_SIGS_01_DS_01,tpvv,No,No,base,512.0,0,0,0,...,4241231.0,5242880.0,NaN,NaN,2021-03-17,mutapsan06,4243968.0,4241230.0,1.24,--
17,79,SIGN_06DC3_T2S_SIGS_01_DS_02,tpvv,No,No,base,512.0,0,0,0,...,4329954.0,5242880.0,NaN,NaN,2021-03-17,mutapsan06,4334208.0,4329954.0,1.21,--
18,73,SNIN_06T2S_ORASNI_BACKUP_01,tpvv,No,No,base,512.0,0,0,0,...,43975.0,46080.0,NaN,NaN,2021-03-17,mutapsan06,47232.0,43974.0,1.05,--
19,69,SNIN_06T2S_ORASNI_DATA_FG2_01,tpvv,No,No,base,512.0,0,0,0,...,173116.0,204800.0,NaN,NaN,2021-03-17,mutapsan06,175104.0,173115.0,1.18,--


In [37]:
# renommage des colonnes
df_3par_volume = df_3par_volume.rename(columns={"Tot_used": "used_capacity", "VSize":"capacity"})

# limitation du nb de colonnes
df_3par_volume = df_3par_volume[["ID","name","capacity","used_capacity","date","pool"]]
df_3par_volume['offre'] = np.array('T2')
#suppression des lignes où la used_capcity est null
df_3par_volume = df_3par_volume[df_3par_volume.used_capacity.notnull()]

# réinitialise les index
df_3par_volume.reset_index(drop=True,inplace =True)

# traitement sur la colonne date
df_3par_volume.date = df_3par_volume.date.map(lambda x :  datetime.datetime.strptime(str(x)[:10],'%Y-%m-%d'))

# ajout d'une colonne origine
df_3par_volume['origine'] = np.array("3par")
df_3par_volume

,ID,name,capacity,used_capacity,date,pool,offre,origine
0,70,GTGN_05T2N_DS_GTG_DATA_02,2301952.0,2301696.0,2018-02-11,mutapsan05,T2,3par
1,69,GTGN_05T2N_DS_GTG_OS_02,512000.0,73984.0,2018-02-11,mutapsan05,T2,3par
2,73,IAXI_05T2S_OVM-12CR2_01,30720.0,32000.0,2018-02-11,mutapsan05,T2,3par
3,44,IAXI_05T2S_OVM-RAC01_01,25600.0,21248.0,2018-02-11,mutapsan05,T2,3par
4,45,IAXI_05T2S_OVM-RAC01_02,25600.0,21248.0,2018-02-11,mutapsan05,T2,3par
...,...,...,...,...,...,...,...,...
47542,78,SIGN_06DC3_T2S_SIGS_01_DS_01,5242880.0,4241231.0,2021-03-17,mutapsan06,T2,3par
47543,79,SIGN_06DC3_T2S_SIGS_01_DS_02,5242880.0,4329954.0,2021-03-17,mutapsan06,T2,3par
47544,73,SNIN_06T2S_ORASNI_BACKUP_01,46080.0,43975.0,2021-03-17,mutapsan06,T2,3par
47545,69,SNIN_06T2S_ORASNI_DATA_FG2_01,204800.0,173116.0,2021-03-17,mutapsan06,T2,3par


In [38]:
# ajout de colonne Client
df_3par_volume['client'] = df_3par_volume.name
df_3par_volume.client = df_3par_volume.client.map(lambda x : x[:3])
df_3par_volume

,ID,name,capacity,used_capacity,date,pool,offre,origine,client
0,70,GTGN_05T2N_DS_GTG_DATA_02,2301952.0,2301696.0,2018-02-11,mutapsan05,T2,3par,GTG
1,69,GTGN_05T2N_DS_GTG_OS_02,512000.0,73984.0,2018-02-11,mutapsan05,T2,3par,GTG
2,73,IAXI_05T2S_OVM-12CR2_01,30720.0,32000.0,2018-02-11,mutapsan05,T2,3par,IAX
3,44,IAXI_05T2S_OVM-RAC01_01,25600.0,21248.0,2018-02-11,mutapsan05,T2,3par,IAX
4,45,IAXI_05T2S_OVM-RAC01_02,25600.0,21248.0,2018-02-11,mutapsan05,T2,3par,IAX
...,...,...,...,...,...,...,...,...,...
47542,78,SIGN_06DC3_T2S_SIGS_01_DS_01,5242880.0,4241231.0,2021-03-17,mutapsan06,T2,3par,SIG
47543,79,SIGN_06DC3_T2S_SIGS_01_DS_02,5242880.0,4329954.0,2021-03-17,mutapsan06,T2,3par,SIG
47544,73,SNIN_06T2S_ORASNI_BACKUP_01,46080.0,43975.0,2021-03-17,mutapsan06,T2,3par,SNI
47545,69,SNIN_06T2S_ORASNI_DATA_FG2_01,204800.0,173116.0,2021-03-17,mutapsan06,T2,3par,SNI


In [39]:
# conversion en GB
df_3par_volume_GB = df_3par_volume.copy()
for colonne in ['capacity','used_capacity']:
    df_3par_volume_GB[colonne] = df_3par_volume_GB[colonne].map(lambda x :en_giga(x))
df_3par_volume_GB

,ID,name,capacity,used_capacity,date,pool,offre,origine,client
0,70,GTGN_05T2N_DS_GTG_DATA_02,2248.0,2247.750000,2018-02-11,mutapsan05,T2,3par,GTG
1,69,GTGN_05T2N_DS_GTG_OS_02,500.0,72.250000,2018-02-11,mutapsan05,T2,3par,GTG
2,73,IAXI_05T2S_OVM-12CR2_01,30.0,31.250000,2018-02-11,mutapsan05,T2,3par,IAX
3,44,IAXI_05T2S_OVM-RAC01_01,25.0,20.750000,2018-02-11,mutapsan05,T2,3par,IAX
4,45,IAXI_05T2S_OVM-RAC01_02,25.0,20.750000,2018-02-11,mutapsan05,T2,3par,IAX
...,...,...,...,...,...,...,...,...,...
47542,78,SIGN_06DC3_T2S_SIGS_01_DS_01,5120.0,4141.827148,2021-03-17,mutapsan06,T2,3par,SIG
47543,79,SIGN_06DC3_T2S_SIGS_01_DS_02,5120.0,4228.470703,2021-03-17,mutapsan06,T2,3par,SIG
47544,73,SNIN_06T2S_ORASNI_BACKUP_01,45.0,42.944336,2021-03-17,mutapsan06,T2,3par,SNI
47545,69,SNIN_06T2S_ORASNI_DATA_FG2_01,200.0,169.058594,2021-03-17,mutapsan06,T2,3par,SNI


In [40]:
## Enregistrement en csv
df_3par_volume_GB.to_csv("df_3par_volume_GB.csv",index=False)

#  3par server

In [41]:
# création du df Server volume avec un fichier server 3par (le plus récent 20191027) 

basepath_3par_server = 'C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/3par/server/lun'
df_3par_server = pd.DataFrame()
for name in ["20210317_mutapsan06_showvlun-lvw.txt", "20210317_mutapsan05_showvlun-lvw.txt"]:
    with open(os.path.join(basepath_3par_server, name)) as f:
        df_temp = pd.read_csv(f, delim_whitespace =True, names =['ID_volume', 'name_volume',"vv_wwn" ,"server" ,"Host_WWN/ISCSI" ,"port","Type","Status", "id"])
        df_temp['date']= name[:8]
        df_3par_server = pd.concat([df_3par_server, df_temp])
df_3par_server.reset_index(drop=True,inplace =True)       
# supprime les lignes où le name n'est pas bon.
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume == "VVName"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume == "VLUNs"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.ID_volume == "VLUN"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume == "total"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server['Host_WWN/ISCSI'] == "----------------"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume.isnull()].index)
df_3par_server

,ID_volume,name_volume,vv_wwn,server,Host_WWN/ISCSI,port,Type,Status,id,date
2,1,GRCS_06DC3_T2S_GRCS_01_DS_01,60002AC0000000000000006F00019826,GRCSVPVIR100,10008666E8200296,0:0:1,host,set,active,20210317
3,1,GRCS_06DC3_T2S_GRCS_01_DS_01,60002AC0000000000000006F00019826,GRCSVPVIR100,10008666E820029A,0:0:2,host,set,active,20210317
4,1,GRCS_06DC3_T2S_GRCS_01_DS_01,60002AC0000000000000006F00019826,GRCSVPVIR100,10008666E8200296,1:0:1,host,set,active,20210317
5,1,GRCS_06DC3_T2S_GRCS_01_DS_01,60002AC0000000000000006F00019826,GRCSVPVIR100,10008666E820029A,1:0:2,host,set,active,20210317
6,1,GRCS_06DC3_T2S_GRCS_01_DS_01,60002AC0000000000000006F00019826,GRCSVPVIR102,10008666E82002A0,0:0:1,host,set,active,20210317
...,...,...,...,...,...,...,...,...,...,...
2408,1,SIGN_05DC1_T2S_SIGS_01_DS_02,60002AC0000000000000007300019825,SIGVPVIR13,21000024FF914893,1:0:2,host,set,active,20210317
2409,1,SIGN_05DC1_T2S_SIGS_01_DS_02,60002AC0000000000000007300019825,SIGVPVIR14,21000024FF9148A6,0:0:1,host,set,active,20210317
2410,1,SIGN_05DC1_T2S_SIGS_01_DS_02,60002AC0000000000000007300019825,SIGVPVIR14,21000024FF9148A7,0:0:2,host,set,active,20210317
2411,1,SIGN_05DC1_T2S_SIGS_01_DS_02,60002AC0000000000000007300019825,SIGVPVIR14,21000024FF9148A6,1:0:1,host,set,active,20210317


In [42]:
df_3par_server['typologie'] = df_3par_server.server
df_3par_server.typologie = df_3par_server.typologie.map(lambda x : typo(x))

In [43]:
# étiquetage de la typologie des volumes 3par
df_3par_volume_plus = df_3par_volume_GB.copy()
liste_typo =[]
for v in df_3par_volume_plus.name:
    if v in df_3par_server.name_volume.unique().tolist():
        liste_typo.append(df_3par_server[df_3par_server.name_volume == v].typologie.values[0])
    else:
        liste_typo.append(typo(v))
        
df_3par_volume_plus['typologie'] = liste_typo
df_3par_volume_plus

,ID,name,capacity,used_capacity,date,pool,offre,origine,client,typologie
0,70,GTGN_05T2N_DS_GTG_DATA_02,2248.0,2247.750000,2018-02-11,mutapsan05,T2,3par,GTG,DAT
1,69,GTGN_05T2N_DS_GTG_OS_02,500.0,72.250000,2018-02-11,mutapsan05,T2,3par,GTG,AUTRE
2,73,IAXI_05T2S_OVM-12CR2_01,30.0,31.250000,2018-02-11,mutapsan05,T2,3par,IAX,VIR
3,44,IAXI_05T2S_OVM-RAC01_01,25.0,20.750000,2018-02-11,mutapsan05,T2,3par,IAX,VIR
4,45,IAXI_05T2S_OVM-RAC01_02,25.0,20.750000,2018-02-11,mutapsan05,T2,3par,IAX,VIR
...,...,...,...,...,...,...,...,...,...,...
47542,78,SIGN_06DC3_T2S_SIGS_01_DS_01,5120.0,4141.827148,2021-03-17,mutapsan06,T2,3par,SIG,VIR
47543,79,SIGN_06DC3_T2S_SIGS_01_DS_02,5120.0,4228.470703,2021-03-17,mutapsan06,T2,3par,SIG,VIR
47544,73,SNIN_06T2S_ORASNI_BACKUP_01,45.0,42.944336,2021-03-17,mutapsan06,T2,3par,SNI,DAT
47545,69,SNIN_06T2S_ORASNI_DATA_FG2_01,200.0,169.058594,2021-03-17,mutapsan06,T2,3par,SNI,DAT


In [44]:
## Enregistrement en csv
df_3par_volume_plus.to_csv("df_3par_volume_plus.csv", index=False)

## Fusion des df volume de IBM et 3par

In [45]:
# union des df ibm et 3PAR
df_volume = pd.concat([df_ibm_volume_plus,df_3par_volume_plus])
df_volume.date = df_volume.date.map(lambda x :  datetime.datetime.strptime(str(x)[:10],'%Y-%m-%d'))
df_volume = df_volume.rename(columns ={'ID':'vdisk_id'})
df_volume

,vdisk_id,name,pool,offre,capacity,used_capacity,date,origine,typologie,client
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC3_T0,T0,10.0,9.830000,2018-10-03,ibm,DAT,INF
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC1_T0,T0,10.0,9.830000,2018-10-03,ibm,DAT,INF
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2,T2,200.0,44.460000,2018-10-03,ibm,DAT,CMC
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2,T2,200.0,44.470000,2018-10-03,ibm,DAT,CMC
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,SDC3_T2,T2,10.0,10.530000,2018-10-03,ibm,DAT,INF
...,...,...,...,...,...,...,...,...,...,...
47542,78,SIGN_06DC3_T2S_SIGS_01_DS_01,mutapsan06,T2,5120.0,4141.827148,2021-03-17,3par,VIR,SIG
47543,79,SIGN_06DC3_T2S_SIGS_01_DS_02,mutapsan06,T2,5120.0,4228.470703,2021-03-17,3par,VIR,SIG
47544,73,SNIN_06T2S_ORASNI_BACKUP_01,mutapsan06,T2,45.0,42.944336,2021-03-17,3par,DAT,SNI
47545,69,SNIN_06T2S_ORASNI_DATA_FG2_01,mutapsan06,T2,200.0,169.058594,2021-03-17,3par,DAT,SNI


In [65]:
## Enregistrement en csv
df_volume[['name','pool','offre','capacity','used_capacity','date','origine','typologie','client']].to_csv("df_volume.csv")

In [66]:
# création d'un df contenant le nom des fichiers récupérés
df_fichiers_stockage_ibm = pd.DataFrame({'nom_fichier':liste_ibm_volume,'origine':np.array('ibm')})
df_fichiers_stockage_3par = pd.DataFrame({'nom_fichier':liste_3par_volume,'origine':np.array('3par')})
df_fichiers_stockage = pd.concat([df_fichiers_stockage_ibm,df_fichiers_stockage_3par])
df_fichiers_stockage

,nom_fichier,origine
0,20181003_mutsapsan019_lssevdiskcopy.txt,ibm
1,20181004_mutsapsan019_lssevdiskcopy.txt,ibm
2,20181005_mutsapsan019_lssevdiskcopy.txt,ibm
3,20181006_mutsapsan019_lssevdiskcopy.txt,ibm
4,20181007_mutsapsan019_lssevdiskcopy.txt,ibm
...,...,...
2245,20210315_mutapsan06_showvv-space.txt,3par
2246,20210316_mutapsan05_showvv-space.txt,3par
2247,20210316_mutapsan06_showvv-space.txt,3par
2248,20210317_mutapsan05_showvv-space.txt,3par


In [67]:
## Enregistrement en csv
df_fichiers_stockage.to_csv("df_fichiers_stockage.csv", index=False)

In [68]:
# Mise en base
df = pd.read_csv('df_volume.csv')
df = df.rename(columns={"name":"name_volume",'pool':'name_pool',"typologie":"name_typologie"})
df

,Unnamed: 0,name_volume,name_pool,offre,capacity,used_capacity,date,origine,name_typologie,client
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC3_T0,T0,10.0,9.830000,2018-10-03,ibm,DAT,INF
1,1,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC1_T0,T0,10.0,9.830000,2018-10-03,ibm,DAT,INF
2,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2,T2,200.0,44.460000,2018-10-03,ibm,DAT,CMC
3,3,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2,T2,200.0,44.470000,2018-10-03,ibm,DAT,CMC
4,4,INFS_19DC1_T2M_TEST_EXLIDAT1_01,SDC3_T2,T2,10.0,10.530000,2018-10-03,ibm,DAT,INF
...,...,...,...,...,...,...,...,...,...,...
681438,47542,SIGN_06DC3_T2S_SIGS_01_DS_01,mutapsan06,T2,5120.0,4141.827148,2021-03-17,3par,VIR,SIG
681439,47543,SIGN_06DC3_T2S_SIGS_01_DS_02,mutapsan06,T2,5120.0,4228.470703,2021-03-17,3par,VIR,SIG
681440,47544,SNIN_06T2S_ORASNI_BACKUP_01,mutapsan06,T2,45.0,42.944336,2021-03-17,3par,DAT,SNI
681441,47545,SNIN_06T2S_ORASNI_DATA_FG2_01,mutapsan06,T2,200.0,169.058594,2021-03-17,3par,DAT,SNI


In [69]:
# pour la création des tables
offres = pd.DataFrame(df.offre.unique(), columns=['offre'])
offres = offres.reset_index()
offres = offres.rename(columns={"index": "id_offre", "offre":"name_offre"})
offres

,id_offre,name_offre
0,0,T0
1,1,T2
2,2,T2_V7000
3,3,T0_V7000
4,4,T2_TMP
5,5,T2_FCM


In [70]:
pools = pd.DataFrame(df.name_pool.unique(), columns=['name_pool'])
pools['name_offre'] = ["T0","T0","T2","T2","T2_V7000","T0_V7000","T2_V7000","T2_TMP","T2_TMP","T2_FCM","T2_FCM","T2","T2"]
pools = pools.reset_index()
pools = pools.rename(columns={"index": "id_pool"})
pools

,id_pool,name_pool,name_offre
0,0,SDC3_T0,T0
1,1,SDC1_T0,T0
2,2,SDC1_T2,T2
3,3,SDC3_T2,T2
4,4,SDC1_T2_V7000,T2_V7000
5,5,SDC3_T0_V7000,T0_V7000
6,6,SDC3_T2_V7000,T2_V7000
7,7,SDC3_T2_TMP,T2_TMP
8,8,SDC1_T2_TMP,T2_TMP
9,9,SDC1_T2_FCM,T2_FCM


In [53]:
pools_plus = pools.merge(offres, on="name_offre")
pools_plus

,id_pool,name_pool,name_offre,id_offre
0,0,SDC3_T0,T0,0
1,1,SDC1_T0,T0,0
2,2,SDC1_T2,T2,1
3,3,SDC3_T2,T2,1
4,11,mutapsan05,T2,1
5,12,mutapsan06,T2,1
6,4,SDC1_T2_V7000,T2_V7000,2
7,6,SDC3_T2_V7000,T2_V7000,2
8,5,SDC3_T0_V7000,T0_V7000,3
9,7,SDC3_T2_TMP,T2_TMP,4


In [54]:
# créer une table typologies et une colonne identifiant
typologies = pd.DataFrame(df.name_typologie.unique(), columns=["name_typologie"])
typologies = typologies.reset_index()
typologies = typologies.rename(columns={"index": "id_typologie"})
typologies

,id_typologie,name_typologie
0,0,DAT
1,1,AUTRE
2,2,VIR
3,3,WEB
4,4,APP


In [55]:
origines = pd.DataFrame(df.origine.unique(), columns=['origine'])
origines = origines.reset_index()
origines = origines.rename(columns={"index": "id_origine"})
origines

,id_origine,origine
0,0,ibm
1,1,3par


In [63]:
df_fichiers_stockage =pd.read_csv('df_fichiers_stockage.csv')
df_fichiers_stockage =df_fichiers_stockage.merge(origines , on="origine")
df_fichiers_stockage

,nom_fichier,origine,id_origine
0,20181003_mutsapsan019_lssevdiskcopy.txt,ibm,0
1,20181004_mutsapsan019_lssevdiskcopy.txt,ibm,0
2,20181005_mutsapsan019_lssevdiskcopy.txt,ibm,0
3,20181006_mutsapsan019_lssevdiskcopy.txt,ibm,0
4,20181007_mutsapsan019_lssevdiskcopy.txt,ibm,0
...,...,...,...
3143,20210315_mutapsan06_showvv-space.txt,3par,1
3144,20210316_mutapsan05_showvv-space.txt,3par,1
3145,20210316_mutapsan06_showvv-space.txt,3par,1
3146,20210317_mutapsan05_showvv-space.txt,3par,1


In [57]:
df = df.merge(pools_plus[["id_pool","name_pool"]], on="name_pool")
df = df.merge(typologies, on="name_typologie")
df = df.merge(origines, on="origine")
df = df.reset_index()
df = df.rename(columns={"index": "id_enreg"})
df

,id_enreg,Unnamed: 0,name_volume,name_pool,offre,capacity,used_capacity,date,origine,name_typologie,client,id_pool,id_typologie,id_origine
0,0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC3_T0,T0,10.0,9.830000,2018-10-03,ibm,DAT,INF,0,0,0
1,1,7,INFS_19DC3_T0M_TEST_EXLIDAT2_01,SDC3_T0,T0,10.0,9.360000,2018-10-03,ibm,DAT,INF,0,0,0
2,2,29,ELEN_19DC3_T0S_ORAD01_DATA_01,SDC3_T0,T0,25.0,0.052246,2018-10-03,ibm,DAT,ELE,0,0,0
3,3,30,ELEN_19DC3_T0S_ORAD01_DATA_02,SDC3_T0,T0,25.0,0.044434,2018-10-03,ibm,DAT,ELE,0,0,0
4,4,31,ELEN_19DC3_T0S_ORAD01_DATA_03,SDC3_T0,T0,25.0,0.036621,2018-10-03,ibm,DAT,ELE,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681438,681438,47359,IPSS_06T2S_SQL_02,mutapsan06,T2,2000.0,1517.434570,2021-03-13,3par,APP,IPS,12,4,1
681439,681439,47401,IPSS_06T2S_SQL_02,mutapsan06,T2,2000.0,1396.427734,2021-03-14,3par,APP,IPS,12,4,1
681440,681440,47443,IPSS_06T2S_SQL_02,mutapsan06,T2,2000.0,1509.578125,2021-03-15,3par,APP,IPS,12,4,1
681441,681441,47485,IPSS_06T2S_SQL_02,mutapsan06,T2,2000.0,1501.647461,2021-03-16,3par,APP,IPS,12,4,1


In [60]:
import psycopg2
import psycopg2.extras as extras

param_dic = {
    "host"      : "localhost",
    "database"  : "cp2",
    "user"      : "postgres",
    "password"  : "Ludi24"
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    print("Connection successful")
    return conn

connection = connect(param_dic)
connection

Connecting to the PostgreSQL database...
Connection successful


<connection object at 0x0000022425C14E18; dsn: 'user=postgres password=xxx dbname=cp2 host=localhost', closed: 0>

In [61]:
def insert_table(conn, df, table):
    """
    Using psycopg2.extras.insert_table() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("insert_table() done")
    cursor.close

In [64]:
# Insert les différents df dans les tables associées
insert_table(connection, origines, 'stk_origines')

insert_table(connection,df_fichiers_stockage[["nom_fichier","id_origine"]], 'stk_fichiers')

insert_table(connection, offres, 'stk_offres')

insert_table(connection, pools_plus[["id_pool","name_pool","id_offre"]], 'stk_pools')

insert_table(connection, typologies, 'stk_typologies')

insert_table(connection, df[['id_enreg','name_volume','capacity','used_capacity', 'date', 'client','id_pool', 'id_typologie', 'id_origine']], 'stk_enregistrements')

insert_table() done
insert_table() done
insert_table() done
insert_table() done
insert_table() done
insert_table() done
